In [1]:
from parameters import get_parameters
from utils_sarimax import *

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t

params = get_parameters()
source_model = params["source_model"]
source_results = params["source_results"]
validation_year = params["validation_year"]
test_year = params["test_year"]


PROJECT_ROOT = Path().resolve().parent
print(f"Project root: {PROJECT_ROOT}")

Project root: C:\Users\dxnin\Documents\dengue-forecast


In [2]:
df = pd.read_pickle(f"{PROJECT_ROOT}/{source_model}/data.pkl")
df = df[["CASES"]]
print(df)

            CASES
DATE             
2007-05-13      0
2007-05-20      0
2007-05-27      0
2007-06-03      0
2007-06-10      0
...           ...
2023-11-26     12
2023-12-03     10
2023-12-10     13
2023-12-17     14
2023-12-24      7

[868 rows x 1 columns]


In [3]:
train_val_df, train_df, val_df, test_df = split_by_date(df, validation_year, test_year)

In [4]:
print("Median validation:", val_df["CASES"].median())
print("Mean validation:", val_df["CASES"].mean())

Median validation: 7.0
Mean validation: 8.426229508196721


In [5]:
print("Median test:", test_df["CASES"].median())
print("Mean test:", test_df["CASES"].mean())

Median test: 3.0
Mean test: 3.5355191256830603


In [ ]:
def diebold_mariano(e1, e2, h=1, loss="rmse"):
    e1, e2 = np.asarray(e1), np.asarray(e2)

    if loss == "rmse":
        d = np.abs(e1) - np.abs(e2)
    elif loss == "mse":
        d = e1**2 - e2**2
    elif loss == "mae":
        d = np.abs(e1) - np.abs(e2)
    elif loss == "mape":
        d = np.abs(e1 / (e1 + e2)) - np.abs(e2 / (e1 + e2))
    else:
        raise ValueError("loss debe ser 'mse', 'mae' o 'mape'")

    d_mean = np.mean(d)
    T = len(d)

    gamma = np.zeros(h)
    for lag in range(h):
        cov = np.sum((d[:T-lag] - d_mean) * (d[lag:] - d_mean))
        gamma[lag] = cov / (T - lag)

    V = gamma[0]
    for lag in range(1, h):
        weight = 1 - lag / h
        V += 2 * weight * gamma[lag]

    DM = d_mean / np.sqrt(V / T)
    p_value = 2 * (1 - t.cdf(abs(DM), df=T - 1))

    return DM, p_value

def compare_models(df1, df2, model1_name, model2_name):
    true1 = df1["Actual"].values
    pred1 = df1["Predicted"].values
    true2 = df2["Actual"].values
    pred2 = df2["Predicted"].values
    
    if not np.allclose(true1, true2):
        print(f"Advertencia: {model1_name} y {model2_name} no tienen el mismo 'Actual'. Se usará el de df1.")

    e1 = true1 - pred1
    e2 = true1 - pred2

    DM, p = diebold_mariano(e1, e2, h=1, loss="rmse")

    if p < 0.05:
        winner = model1_name if DM < 0 else model2_name
    else:
        winner = "No significativo"

    return DM, p, winner


def load_results(PROJECT_ROOT, source_results):
    paths = {
        "LSTM_val":      f"{PROJECT_ROOT}/{source_results}/LSTM/test/true_pred.pkl",
        "LSTM_exog_val": f"{PROJECT_ROOT}/{source_results}/LSTM_exog/test/true_pred.pkl",
        "SARIMAX_val":      f"{PROJECT_ROOT}/{source_results}/SARIMAX/test/true_pred.pkl",
        "SARIMAX_exog_val": f"{PROJECT_ROOT}/{source_results}/SARIMAX_exog/test/true_pred.pkl",

        "LSTM_test":      f"{PROJECT_ROOT}/{source_results}/LSTM/validation/best_true_pred.pkl",
        "LSTM_exog_test": f"{PROJECT_ROOT}/{source_results}/LSTM_exog/validation/best_true_pred.pkl",
        "SARIMAX_test":      f"{PROJECT_ROOT}/{source_results}/SARIMAX/validation/best_true_pred.pkl",
        "SARIMAX_exog_test": f"{PROJECT_ROOT}/{source_results}/SARIMAX_exog/validation/best_true_pred.pkl"
    }

    dfs = {name: pd.read_pickle(path) for name, path in paths.items()}
    return dfs


def run_dm_analysis(PROJECT_ROOT, source_results):

    dfs = load_results(PROJECT_ROOT, source_results)

    results = {
        "SARIMAX validation": compare_models(
            dfs["SARIMAX_val"], dfs["SARIMAX_exog_val"], "SARIMAX", "SARIMAX_exog"
        ),
        "SARIMAX test": compare_models(
            dfs["SARIMAX_test"], dfs["SARIMAX_exog_test"], "SARIMAX", "SARIMAX_exog"
        ),
        "LSTM validation": compare_models(
            dfs["LSTM_val"], dfs["LSTM_exog_val"], "LSTM", "LSTM_exog"
        ),
        "LSTM test": compare_models(
            dfs["LSTM_test"], dfs["LSTM_exog_test"], "LSTM", "LSTM_exog"
        ),
    }

    table = pd.DataFrame({
        col: {
            "DM statistic": results[col][0],
            "p-value (%)": results[col][1],
            "Ganador": results[col][2]
        }
        for col in results
    })

    return table.T


table = run_dm_analysis(PROJECT_ROOT, source_results)
print(table)

                   DM statistic p-value (%)           Ganador
SARIMAX validation    -0.792569    0.429061  No significativo
SARIMAX test          -0.989632    0.323668  No significativo
LSTM validation       -1.144836     0.25378  No significativo
LSTM test             -0.135443    0.892411  No significativo


In [7]:
def compare_with_naive(naive_df, model_df, model_name, dataset_name):
    true_n = naive_df["Actual"].values
    pred_n = naive_df["Predicted"].values
    
    true_m = model_df["Actual"].values
    pred_m = model_df["Predicted"].values

    if not np.allclose(true_n, true_m):
        print(f"Advertencia: Naive y {model_name} no tienen el mismo 'Actual'. Se usa el de Naive.")

    e1 = true_n - pred_n  
    e2 = true_n - pred_m  

    DM, p = diebold_mariano(e1, e2, h=1, loss="rmse")

    if p < 0.05:
        winner = "Naive" if DM < 0 else model_name
    else:
        winner = "No significativo"

    return {
        "models": f"Naive vs {model_name}",
        "dataset": dataset_name,
        "DM statistic": DM,
        "p-value (%)": p,
        "Ganador": winner
    }


def load_results(PROJECT_ROOT, source_results):
    paths = {
        "LSTM_val": f"{PROJECT_ROOT}/{source_results}/LSTM/test/true_pred.pkl",
        "LSTM_test": f"{PROJECT_ROOT}/{source_results}/LSTM/validation/best_true_pred.pkl",

        "LSTM_exog_val": f"{PROJECT_ROOT}/{source_results}/LSTM_exog/test/true_pred.pkl",
        "LSTM_exog_test": f"{PROJECT_ROOT}/{source_results}/LSTM_exog/validation/best_true_pred.pkl",

        "SARIMAX_val": f"{PROJECT_ROOT}/{source_results}/SARIMAX/test/true_pred.pkl",
        "SARIMAX_test": f"{PROJECT_ROOT}/{source_results}/SARIMAX/validation/best_true_pred.pkl",

        "SARIMAX_exog_val": f"{PROJECT_ROOT}/{source_results}/SARIMAX_exog/test/true_pred.pkl",
        "SARIMAX_exog_test": f"{PROJECT_ROOT}/{source_results}/SARIMAX_exog/validation/best_true_pred.pkl",
    }

    dfs = {name: pd.read_pickle(path) for name, path in paths.items()}

    actual_val = dfs["LSTM_val"]["Actual"].copy().reset_index(drop=True)
    naive_pred_val = actual_val.shift(1)
    naive_pred_val.iloc[0] = actual_val.iloc[0]

    dfs["Naive_val"] = pd.DataFrame({
        "Actual": actual_val,
        "Predicted": naive_pred_val
    })

    actual_test = dfs["LSTM_test"]["Actual"].copy().reset_index(drop=True)
    naive_pred_test = actual_test.shift(1)
    naive_pred_test.iloc[0] = actual_test.iloc[0]

    dfs["Naive_test"] = pd.DataFrame({
        "Actual": actual_test,
        "Predicted": naive_pred_test
    })

    return dfs



def run_dm_naive(PROJECT_ROOT, source_results):

    dfs = load_results(PROJECT_ROOT, source_results)

    rows = []

    rows.append(compare_with_naive(dfs["Naive_val"], dfs["SARIMAX_val"], "SARIMAX", "Validation"))
    rows.append(compare_with_naive(dfs["Naive_val"], dfs["SARIMAX_exog_val"], "SARIMAX Climate", "Validation"))
    rows.append(compare_with_naive(dfs["Naive_val"], dfs["LSTM_val"], "LSTM", "Validation"))
    rows.append(compare_with_naive(dfs["Naive_val"], dfs["LSTM_exog_val"], "LSTM Climate", "Validation"))

    rows.append(compare_with_naive(dfs["Naive_test"], dfs["SARIMAX_test"], "SARIMAX", "Test"))
    rows.append(compare_with_naive(dfs["Naive_test"], dfs["SARIMAX_exog_test"], "SARIMAX Climate", "Test"))
    rows.append(compare_with_naive(dfs["Naive_test"], dfs["LSTM_test"], "LSTM", "Test"))
    rows.append(compare_with_naive(dfs["Naive_test"], dfs["LSTM_exog_test"], "LSTM Climate", "Test"))

    table = pd.DataFrame(rows)
    return table


table = run_dm_naive(PROJECT_ROOT, source_results)
print(table)

                     models     dataset  DM statistic  p-value (%)  \
0          Naive vs SARIMAX  Validation     -0.089562     0.928734   
1  Naive vs SARIMAX Climate  Validation     -0.337110     0.736422   
2             Naive vs LSTM  Validation      3.350175     0.000982   
3     Naive vs LSTM Climate  Validation      2.917147     0.003977   
4          Naive vs SARIMAX        Test     -1.969385     0.050427   
5  Naive vs SARIMAX Climate        Test     -2.202209     0.028907   
6             Naive vs LSTM        Test      3.082556     0.002372   
7     Naive vs LSTM Climate        Test      2.167237     0.031516   

            Ganador  
0  No significativo  
1  No significativo  
2              LSTM  
3      LSTM Climate  
4  No significativo  
5             Naive  
6              LSTM  
7      LSTM Climate  
